In [2]:
import sys
sys.path.append('/home/ak287/seisbench/seisbench/models')
import seisbench.models as sbm

In [3]:
import seisbench
print(seisbench.__file__)  # should now point to ~/seisbench/seisbench/__init__.py

import seisbench.models as sbm
print(hasattr(sbm, "SeismicCNN"))  # should print True


/home/ak287/seisbench/seisbench/__init__.py
True


In [4]:
from seismiccnn import SeismicCNN

In [5]:
import seisbench.models as sbm
print(hasattr(sbm, "SeismicCNN"))


True


In [6]:
#model1 = sbm.QuakeXNetoneD.from_pretrained("base")
model2 = sbm.SeismicCNN.from_pretrained("base")